# OpenET DisALEXI
## Collection "Interpolate" Examples

In [1]:
import datetime
import pprint

import ee
import pandas as pd

from IPython.display import Image
import openet.disalexi as model

ee.Initialize()

In [2]:
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

## Input parameters

In [3]:
collections = ['LANDSAT/LC08/C01/T1_SR']

etr_source = 'projects/climate-engine/cimis/daily'
etr_band = 'ETr_ASCE'
# etr_source = 'IDAHO_EPSCOR/GRIDMET'
# etr_band = 'etr'

# Date range you want to aggregate ET over
# End date is inclusive (like filterDate() calls)
start_date = '2017-07-01'
end_date = '2017-08-01'

# Only keep images with an average cloud cover less than
# Cloud cover filter parameter is not being passed in (yet)
cloud_cover = 70

# Number of extra days (at start and end) to include in interpolation
interp_days = 32
# Interpolation method - currently only LINEAR is supported
interp_method = 'LINEAR'

test_point = ee.Geometry.Point(-121.5265, 38.7399)
study_area = ee.Geometry.Rectangle(-122.00, 38.50, -121.00, 39.0)

# Hard code the study area and CRS
study_region = study_area.bounds(1, 'EPSG:4326').coordinates().getInfo()
study_crs = 'EPSG:32610'

## Build the collection object

In [4]:
model_obj = model.Collection(
    collections=collections,
    etr_source=etr_source, 
    etr_band=etr_band,
    start_date=start_date,
    end_date=end_date,
    geometry=test_point,
    cloud_cover_max=70,
    # filter_args={},
)

This isn't returning the images used for interpolation.  Should it?

In [5]:
pprint.pprint(model_obj.get_image_ids())

['LANDSAT/LC08/C01/T1_SR/LC08_044033_20170716']


In [6]:
def get_region_df(info):
    """Convert the output of getRegions to a pandas dataframe"""
    col_dict = {}
    info_dict = {}
    for i, k in enumerate(info[0][4:]):
        col_dict[k] = i+4
        info_dict[k] = {}
        
    for row in info[1:]:
        date = datetime.datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')
        for k, v in col_dict.items():
            info_dict[k][date] = row[col_dict[k]]
            
    return pd.DataFrame.from_dict(info_dict)

## Interpolate Daily ET

In [7]:
et_daily_coll = model_obj.interpolate(
    t_interval='daily', 
    variables=['et', 'etr', 'etf'], 
    interp_method=interp_method,
    interp_days=interp_days,
)
et_daily_df = get_region_df(et_daily_coll.getRegion(test_point, scale=30).getInfo())
pprint.pprint(et_daily_df)
print('')
pprint.pprint(et_daily_df[['et', 'etr']].sum())

EEException: User memory limit exceeded.

## Interpolate Monthly ET

In [ ]:
et_monthly_coll = model_obj.interpolate(
    t_interval='monthly', 
    variables=['et', 'etr', 'etf'], 
    interp_method=interp_method,
    interp_days=interp_days,
)
et_monthly_df = get_region_df(et_monthly_coll.getRegion(test_point, scale=30).getInfo())
pprint.pprint(et_monthly_df)

## Sum of Daily ET (for one month)

In [ ]:
Image(url=ee.Image(et_daily_coll.select(['et']).sum())\
        .reproject(crs=study_crs, scale=100)\
        .getThumbURL({'min': 0.0, 'max': 300, 'palette': et_palette, 'region': study_region}),
      embed=True, format='png') 

## Monthly ET (for one month)

In [ ]:
Image(url=ee.Image(et_monthly_coll.select(['et']).sum())\
        .reproject(crs=study_crs, scale=100)\
        .getThumbURL({'min': 0.0, 'max': 300, 'palette': et_palette, 'region': study_region}),
      embed=True, format='png') 

## Monthly ETr (for one month)

In [ ]:
Image(url=ee.Image(et_monthly_coll.select(['etr']).sum())\
        .reproject(crs=study_crs, scale=100)\
        .getThumbURL({'min': 0.0, 'max': 300, 'palette': et_palette, 'region': study_region}),
      embed=True, format='png') 